In [ ]:
file_path = "/content/EEG_Scaled_data.csv"
df_sample = pd.read_csv(file_path, nrows=5)
print("Colonnes du dataset:", df_sample.columns)
print(df_sample.head())

Colonnes du dataset: Index(['Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5',
       'Channel_6', 'Channel_7', 'Channel_8', 'Channel_9', 'Channel_10',
       ...
       'Channel_36856', 'Channel_36857', 'Channel_36858', 'Channel_36859',
       'Channel_36860', 'Channel_36861', 'Channel_36862', 'Channel_36863',
       'Channel_36864', 'target'],
      dtype='object', length=36865)
   Channel_1  Channel_2  Channel_3  Channel_4  Channel_5  Channel_6  \
0      0.647      0.149     -0.213     -0.199     -0.287     -0.187   
1     -2.450     -2.788     -2.387     -1.370     -1.032     -1.037   
2     -0.026     -0.123     -0.347     -0.348      0.027      0.162   
3     -0.067     -0.153     -0.180     -0.210     -0.238     -0.183   
4     -0.190     -0.299     -0.333     -0.199      0.182      0.261   

   Channel_7  Channel_8  Channel_9  Channel_10  ...  Channel_36856  \
0      0.320      0.445      0.482       0.348  ...          0.076   
1     -1.253     -1.702     -2.116 

In [ ]:
chunk_size = 1000
import pandas as pd


In [ ]:
# Paramètres
file_path = "/content/EEG_Scaled_data.csv"  # Chemin sur Colab
chunk_size = 1000  # Lire petit à petit (ajuster si besoin)
window_size = 256  # Taille de la fenêtre (1 seconde si 256 Hz)
step_size = 256    # Pas de la fenêtre (peut être < window_size pour du overlapping)
target_col = "target"

In [ ]:
import numpy as np

def segment_chunk(chunk, window_size=256, step_size=256):
    X_segments = []
    y_segments = []

    data = chunk.drop(columns=[target_col]).values
    targets = chunk[target_col].values

    for i in range(0, len(chunk) - window_size + 1, step_size):
        segment = data[i:i+window_size]
        label_segment = targets[i:i+window_size]

        # On donne le label 1 à la fenêtre si au moins un point est une crise
        label = 1 if np.any(label_segment == 1) else 0

        X_segments.append(segment)
        y_segments.append(label)

    return X_segments, y_segments


In [ ]:
X_all, y_all = [], []

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    X_seg, y_seg = segment_chunk(chunk, window_size, step_size)
    X_all.extend(X_seg)
    y_all.extend(y_seg)


In [ ]:
print("Nombre total de segments générés:", len(X_all))
print("Nombre de segments de crise:", sum(y_all))
print("Nombre de segments sans crise:", len(y_all) - sum(y_all))



Nombre total de segments générés: 33
Nombre de segments de crise: 25
Nombre de segments sans crise: 8


In [ ]:
# Vérifier combien de lignes totales sont associées à une crise
count_0, count_1 = 0, 0
for chunk in pd.read_csv(file_path, usecols=[target_col], chunksize=50000):
    count_0 += (chunk[target_col] == 0).sum()
    count_1 += (chunk[target_col] == 1).sum()

print(f"Lignes sans crise: {count_0}, Lignes avec crise: {count_1}")


Lignes sans crise: 9799, Lignes avec crise: 1434


In [ ]:
import pandas as pd
import numpy as np
import os

# Paramètres
file_path = "/content/EEG_Scaled_data.csv"  # adapte si nécessaire
target_col = "target"
window_size = 256
step_size = 64
output_dir = "/content/segments_npz"

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_dir, exist_ok=True)

# Variables pour le suivi
segment_count = 0
crisis_count = 0
non_crisis_count = 0
buffer = pd.DataFrame()

# Lire le fichier par chunks
for chunk in pd.read_csv(file_path, chunksize=1000):
    buffer = pd.concat([buffer, chunk], ignore_index=True)

    # Tant qu'on a assez de lignes pour une fenêtre
    while len(buffer) >= window_size:
        segment = buffer.iloc[:window_size]
        label = 1 if segment[target_col].sum() > 0 else 0

        # Sauvegarde du segment
        filename = f"segment_{segment_count:05d}.npz"
        np.savez_compressed(os.path.join(output_dir, filename),
                            data=segment.drop(columns=[target_col]).to_numpy(),
                            label=label)

        # Compter
        segment_count += 1
        if label == 1:
            crisis_count += 1
        else:
            non_crisis_count += 1

        # Supprimer les lignes utilisées en fonction du step
        buffer = buffer.iloc[step_size:].reset_index(drop=True)

print(f"Segments générés : {segment_count}")
print(f"  → Avec crise : {crisis_count}")
print(f"  → Sans crise : {non_crisis_count}")


Segments générés : 172
  → Avec crise : 139
  → Sans crise : 33


In [ ]:
import numpy as np
from glob import glob

segment_files = glob("/content/segments_npz/*.npz")
for file in segment_files[:5]:  # afficher les 5 premiers
    data = np.load(file)
    print("Shape:", data['data'].shape, "Label:", data['label'])


Shape: (256, 36864) Label: 1
Shape: (256, 36864) Label: 0
Shape: (256, 36864) Label: 0
Shape: (256, 36864) Label: 1
Shape: (256, 36864) Label: 0
